In [1]:
import os
import geopandas as gpd
import pandas as pd

In [2]:
import sys
sys.path.append('../')
from process.append import get_enriched_features

In [3]:
california_boundary = gpd.read_file(r'D:\WORK\wildfire\Interagency-Tracking-System\its\Interagency Tracking System.gdb', 
                                    driver='OpenFileGDB', 
                                    layer='California')

C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(
C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: organizePolygons() received a polygon with more than 100 parts. The processing may be really slow.  You can skip the processing by setting METHOD=SKIP, or only make it analyze counter-clock wise parts by setting METHOD=ONLY_CCW if you can assume that the outline of holes is counter-clock wise defined
  return ogr_read(


In [4]:
enriched_path = r"D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0"

In [5]:
enriched_list = os.listdir(enriched_path)

In [6]:
append_path = r"D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0\appended.gdb"

In [7]:
report_path = r'D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0\reports.gdb'

In [8]:
# skip the append, reports gdb file path
skip_list = ['appended.gdb', 'reports.gdb']

In [9]:
point_layers = []
line_layers = []
poly_layers = []
for f in enriched_list:
    if f in skip_list:
        continue
    f_path = enriched_path + r"\{}".format(f)
    gdb_layers = gpd.list_layers(f_path)
    for i in range(len(gdb_layers)):
        if 'point' in gdb_layers.loc[i, 'geometry_type'].lower():
            point_layers.append({'gdb_path': f_path, 'layer_name':gdb_layers.loc[i, 'name']})
        elif 'line' in gdb_layers.loc[i, 'geometry_type'].lower():
            line_layers.append({'gdb_path': f_path, 'layer_name':gdb_layers.loc[i, 'name']})
        else:
            poly_layers.append({'gdb_path': f_path, 'layer_name':gdb_layers.loc[i, 'name']})
            
enriched_layers = {'point': point_layers, 
                  'line': line_layers,
                  'polygon': poly_layers}

In [11]:
# one time change for activity category

In [18]:
from utils.category import categorize_activity
from utils.standardize_domains import standardize_domains
from utils.counts_to_mas import counts_to_mas

In [19]:
for lyr_type in enriched_layers.keys():
    for path_dict in enriched_layers[lyr_type]:
        gdf = gpd.read_file(path_dict['gdb_path'], driver='OpenFileGDB', layer=path_dict['layer_name'])
        gdf = counts_to_mas(standardize_domains(categorize_activity(gdf)), 1950, 2025)

        #if ('BLM' in path_dict['layer_name']) or ('NFPORS' in path_dict['layer_name']) or ('NPS' in path_dict['layer_name']):
        #    print(path_dict['layer_name'])
        #    gdf.loc[gdf['ACTIVITY_START'] >= f'2023-10-01', 'COUNTS_TO_MAS'] = 'NO'
        print(path_dict['layer_name'])
        gdf.to_file(path_dict['gdb_path'], driver='OpenFileGDB', layer=path_dict['layer_name'])
    

C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(
2025-07-14 12:33:50,702 INFO  [utils.counts_to_mas   ]           Calculating Counts to MAS
2025-07-14 12:33:50,703 INFO  [utils.counts_to_mas   ]              counts step 1/8: set to 'NO'
2025-07-14 12:33:50,703 INFO  [utils.counts_to_mas   ]              counts step 2/8: select by bounding years (1950-2025)
2025-07-14 12:33:50,706 INFO  [utils.counts_to_mas   ]              counts step 3/8: set to 'YES' if activity description is in the list
2025-07-14 12:33:50,706 INFO  [utils.counts_to_mas   ]              counts step 4/8: set to 'NO' if not 'Acres'
2025-07-14 12:33:50,707 INFO  [utils.counts_to_mas   ]              counts step 5/8: set to 'NO' if status is 'Canceled', 'Planned', 'Outyear', or 'Proposed'
2025-07-14 12:33:50,708 INFO  [utils.counts_to_mas   ]              counts step 6/8: set to 'NO' if Activity Category i

NOT_DEFINED
OTHER_FUELS_REDUCTION
NOT_DEFINED
OTHER_FUELS_REDUCTION
NOT_DEFINED
OTHER_FUELS_REDUCTION
NOT_DEFINED
OTHER_FUELS_REDUCTION
NOT_DEFINED
OTHER_FUELS_REDUCTION
NOT_DEFINED
OTHER_FUELS_REDUCTION
None
NOT_DEFINED
None
OTHER_FUELS_REDUCTION
None
OTHER_FUELS_REDUCTION
None
FOREST_STEWARDSHIP
None
OTHER_FOREST_MGMT
None
OTHER_FOREST_MGMT
CNRA_enriched_20250626_point


2025-07-14 12:33:50,878 INFO  [pyogrio._io           ]  Created 2,066 records
C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(
2025-07-14 12:33:51,227 INFO  [utils.counts_to_mas   ]           Calculating Counts to MAS
2025-07-14 12:33:51,227 INFO  [utils.counts_to_mas   ]              counts step 1/8: set to 'NO'
2025-07-14 12:33:51,228 INFO  [utils.counts_to_mas   ]              counts step 2/8: select by bounding years (1950-2025)
2025-07-14 12:33:51,230 INFO  [utils.counts_to_mas   ]              counts step 3/8: set to 'YES' if activity description is in the list
2025-07-14 12:33:51,231 INFO  [utils.counts_to_mas   ]              counts step 4/8: set to 'NO' if not 'Acres'
2025-07-14 12:33:51,232 INFO  [utils.counts_to_mas   ]              counts step 5/8: set to 'NO' if status is 'Canceled', 'Planned', 'Outyear', or 'Proposed'
2025-07-14 12:33:51,232 INFO  [utils.coun

NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
None
None
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
None
None
None
None
None
None
None
None
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
NOT_DEFINED
OTHER_FUELS_REDUCTION
None
None
None
None
None
None
None
None
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
None
None
None
None
None
None
None
None
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
None
None
None
None
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
None
None
None
None
None
None
None
None
None
None
None
None
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
None
None
NOT_DEFINED
NOT_DEFINED
None
None
None
None
None
None
None
None
None
None
NOT_DEFINED
NOT_DEFINED
None
None
None
None
None
None
None
None
NOT_DEFINED
NO

2025-07-14 12:33:51,486 INFO  [pyogrio._io           ]  Created 4,555 records
C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(
2025-07-14 12:33:52,491 INFO  [utils.counts_to_mas   ]           Calculating Counts to MAS
2025-07-14 12:33:52,491 INFO  [utils.counts_to_mas   ]              counts step 1/8: set to 'NO'
2025-07-14 12:33:52,491 INFO  [utils.counts_to_mas   ]              counts step 2/8: select by bounding years (1950-2025)
2025-07-14 12:33:52,496 INFO  [utils.counts_to_mas   ]              counts step 3/8: set to 'YES' if activity description is in the list
2025-07-14 12:33:52,498 INFO  [utils.counts_to_mas   ]              counts step 4/8: set to 'NO' if not 'Acres'
2025-07-14 12:33:52,500 INFO  [utils.counts_to_mas   ]              counts step 5/8: set to 'NO' if status is 'Canceled', 'Planned', 'Outyear', or 'Proposed'
2025-07-14 12:33:52,501 INFO  [utils.coun

...

NOT_DEFINED
REFORESTATION
NOT_DEFINED
None
NOT_DEFINED
REFORESTATION
NOT_DEFINED
None
NOT_DEFINED
REFORESTATION
NOT_DEFINED
None
NOT_DEFINED
None
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
REFORESTATION
NOT_DEFINED
REFORESTATION
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
REFORESTATION
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
REFORESTATION
None
None
NOT_DEFINED
REFORESTATION
NOT_DEFINED
REFORESTATION
NOT_DEFINED
REFORESTATION
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
REFORESTATION
NOT_DEFINED
REFORESTATION
NOT_DEFINED
None
NOT_DEFINED
REFORESTATION
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
REFORESTATION
NOT_DEFINED
REFORESTATION
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
REFORESTATION
NOT_DEFINED
REFORESTATION
NOT_DEFINED
REFORESTATION
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
REFORESTATION
NOT_DEFINED
REFORESTATION
NOT_DEFINED
REFORESTATION
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
REFORESTATION
NOT_DEFINED
REFORESTATION
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
REFORESTATION
NOT_DEFINED
REFORESTATION
NOT_DEFINED
REFORESTATION
NOT_DEFINED
R

2025-07-14 12:35:21,614 INFO  [utils.counts_to_mas   ]           Calculating Counts to MAS
2025-07-14 12:35:21,615 INFO  [utils.counts_to_mas   ]              counts step 1/8: set to 'NO'
2025-07-14 12:35:21,619 INFO  [utils.counts_to_mas   ]              counts step 2/8: select by bounding years (1950-2025)
2025-07-14 12:35:21,634 INFO  [utils.counts_to_mas   ]              counts step 3/8: set to 'YES' if activity description is in the list
2025-07-14 12:35:21,677 INFO  [utils.counts_to_mas   ]              counts step 4/8: set to 'NO' if not 'Acres'
2025-07-14 12:35:21,713 INFO  [utils.counts_to_mas   ]              counts step 5/8: set to 'NO' if status is 'Canceled', 'Planned', 'Outyear', or 'Proposed'
2025-07-14 12:35:21,761 INFO  [utils.counts_to_mas   ]              counts step 6/8: set to 'NO' if Activity Category is 'Watershed Improvement'
2025-07-14 12:35:21,780 INFO  [utils.counts_to_mas   ]              counts step 7/8: set to 'NO' if Agency is 'Other' and Admin is 'CARB'


USFS_Region05_enriched_20250623


2025-07-14 12:35:40,971 INFO  [pyogrio._io           ]  Created 413,214 records
C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(
2025-07-14 12:35:46,776 INFO  [utils.counts_to_mas   ]           Calculating Counts to MAS
2025-07-14 12:35:46,776 INFO  [utils.counts_to_mas   ]              counts step 1/8: set to 'NO'
2025-07-14 12:35:46,777 INFO  [utils.counts_to_mas   ]              counts step 2/8: select by bounding years (1950-2025)
2025-07-14 12:35:46,777 INFO  [utils.counts_to_mas   ]              counts step 3/8: set to 'YES' if activity description is in the list
2025-07-14 12:35:46,778 INFO  [utils.counts_to_mas   ]              counts step 4/8: set to 'NO' if not 'Acres'
2025-07-14 12:35:46,778 INFO  [utils.counts_to_mas   ]              counts step 5/8: set to 'NO' if status is 'Canceled', 'Planned', 'Outyear', or 'Proposed'
2025-07-14 12:35:46,779 INFO  [utils.co

NOT_DEFINED
REFORESTATION
None
None
None
None
NOT_DEFINED
REFORESTATION
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
USFS_Region06_enriched_20250623


In [12]:
enriched_polygons, enriched_lines, enriched_points = get_enriched_features(enriched_layers)

2025-07-18 12:29:23,241 INFO  [process.append_polygon]  --------------------------------------------------------------------------------
2025-07-18 12:29:23,241 INFO  [process.append_polygon]  Concatenate all polygon records
2025-07-18 12:29:23,241 INFO  [process.append_polygon]  Load GeoDataFrame from the layer 'BLM_enriched_20250710' in 'D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0\BLM_1950_2025.gdb' 
C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(
2025-07-18 12:29:23,331 INFO  [process.append_polygon]  Load GeoDataFrame from the layer 'CNRA_enriched_20250626_polygon' in 'D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0\CNRA_1950_2025.gdb' 
C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(
2025-07-18 12:29:27,195 INF

In [13]:
import dask_geopandas
# some data need to be converted to multi-type again
from utils.save_gdf_to_gdb import save_gdf_to_gdb

In [14]:
# grab timber non spatial path again
timber_nonspatial_path = None
for p in enriched_layers['point']:
    if 'Timber_Nonspatial' in p['gdb_path']:
        timber_nonspatial_path = p
        break

In [15]:
timber_nonspatial = gpd.read_file(timber_nonspatial_path['gdb_path'], 
                                    driver='OpenFileGDB', 
                                    layer=timber_nonspatial_path['layer_name'])

C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(


In [16]:
%%time
for df, lyr_name in zip([enriched_polygons,enriched_lines,enriched_points], ["appended_poly","appended_line","appended_point"]):
    # init dask gdf for multithread clipping
    ddf = dask_geopandas.from_geopandas(df, npartitions=16)
    # clip to california bounds
    append_clipped = ddf.sjoin(california_boundary, how='inner', predicate='intersects').compute()
    # drop unwanted artifact columns from California boundary df
    append_clipped = append_clipped.drop(['index_right', 'Shape_Area', 'Shape_Length'], axis=1)
    
    # industry nonspatial is by design out of california bounds and got clipped, manually concat it back
    if lyr_name == 'appended_point':
        append_clipped = pd.concat([append_clipped, timber_nonspatial], ignore_index=True)
    save_gdf_to_gdb(append_clipped, append_path, lyr_name)

2025-07-18 12:36:36,252 INFO  [utils.save_gdf_to_gdb ]        Windows machine detected
2025-07-18 12:36:36,253 INFO  [utils.save_gdf_to_gdb ]        Check geodataframe geometry object and cast to Multi-x type if both exist
2025-07-18 12:36:36,283 INFO  [utils.save_gdf_to_gdb ]        Running GDAL OpenFileGDB to save to file


D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0\appended.gdb


2025-07-18 12:37:05,701 INFO  [pyogrio._io           ]  Created 455,318 records
2025-07-18 12:37:12,970 INFO  [utils.save_gdf_to_gdb ]        File saved to D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0\appended.gdb appended_poly
2025-07-18 12:39:34,470 INFO  [utils.save_gdf_to_gdb ]        Windows machine detected
2025-07-18 12:39:34,470 INFO  [utils.save_gdf_to_gdb ]        Check geodataframe geometry object and cast to Multi-x type if both exist
2025-07-18 12:39:34,479 INFO  [utils.save_gdf_to_gdb ]        Running GDAL OpenFileGDB to save to file


D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0\appended.gdb


2025-07-18 12:39:54,400 INFO  [pyogrio._io           ]  Created 131,658 records
2025-07-18 12:39:58,549 INFO  [utils.save_gdf_to_gdb ]        File saved to D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0\appended.gdb appended_line
<timed exec>:11: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
<timed exec>:11: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
<timed exec>:11: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future

D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0\appended.gdb


2025-07-18 12:40:01,529 INFO  [pyogrio._io           ]  Created 20,443 records
2025-07-18 12:40:01,678 INFO  [utils.save_gdf_to_gdb ]        File saved to D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0\appended.gdb appended_point


CPU times: total: 46min 22s
Wall time: 9min 58s


In [17]:
append_path = r"D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0\appended.gdb"

In [18]:
gpd.list_layers(append_path)

,name,geometry_type
0,appended_poly,MultiPolygon
1,appended_line,MultiLineString
2,appended_point,Point


In [19]:
enriched_polygons = gpd.read_file(append_path, driver='OpenFileGDB', layer='appended_poly')
enriched_points = gpd.read_file(append_path, driver='OpenFileGDB', layer='appended_point')
enriched_lines = gpd.read_file(append_path, driver='OpenFileGDB', layer='appended_line')

C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(
C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(
C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(


In [20]:
enriched_points[(enriched_points.AGENCY =='<NA>')].PROJECTID_USER.apply(lambda x: x[:5]).unique()

array([], dtype=object)

In [21]:
enriched_polygons[(enriched_polygons.COUNTS_TO_MAS=="YES") & (enriched_polygons.ADMINISTERING_ORG.isna())]

,PROJECTID_USER,AGENCY,ORG_ADMIN_p,PROJECT_CONTACT,PROJECT_EMAIL,ADMINISTERING_ORG,PROJECT_NAME,PROJECT_STATUS,PROJECT_START,PROJECT_END,...,Federal_FY,State_FY,TRMT_GEOM,COUNTS_TO_MAS,FIPS,STATE,ST_ABBREV,SQMI,POPULATION,geometry
35880,CATNR-HFR-FY24-MX-W,None,None,None,None,None,CATNR-HFR-FY24-MX-W,None,None,None,...,2024,2025,POLYGON,YES,06,California,CA,158144.83,39648525,"MULTIPOLYGON (((294989.891 -581637.748, 294979..."
35881,CASOR-HFR-FY24-RX-N,None,None,None,None,None,CASOR-HFR-FY24-RX-N,None,None,None,...,2024,2024,POLYGON,YES,06,California,CA,158144.83,39648525,"MULTIPOLYGON (((412018.092 -526655.842, 412041..."
35882,CASOR-HFR-FY24-MX-N,None,None,None,None,None,CASOR-HFR-FY24-MX-N,None,None,None,...,2024,2025,POLYGON,YES,06,California,CA,158144.83,39648525,"MULTIPOLYGON (((411839.652 -526745.964, 411831..."
35886,NPS PWR SAMO FY24 Strategic Fuels - BIL,None,None,None,None,None,NPS PWR SAMO FY24 Strategic Fuels - BIL,None,None,None,...,2024,2025,POLYGON,YES,06,California,CA,158144.83,39648525,"MULTIPOLYGON (((115581.03 -433029.251, 115591...."
35887,CHIS FY24 Rehab and monitor burn pile areas - BIL,None,None,None,None,None,CHIS FY24 Rehab and monitor burn pile areas - BIL,None,None,None,...,2024,2025,POLYGON,YES,06,California,CA,158144.83,39648525,"MULTIPOLYGON (((29569.069 -443998.055, 29562.4..."
35904,Oak Mountain Dozer Maintenance Fuel Break,None,None,None,None,None,Oak Mountain Dozer Maintenance Fuel Break,None,None,None,...,2024,2024,POLYGON,YES,06,California,CA,158144.83,39648525,"MULTIPOLYGON (((107210.688 -222139.575, 107214..."
35905,Water Plant Dozer line 1,None,None,None,None,None,Water Plant Dozer line 1,None,None,None,...,2024,2024,POLYGON,YES,06,California,CA,158144.83,39648525,"MULTIPOLYGON (((105684.906 -221627.456, 105687..."
35906,South Reservation Handline 1 Maintenance,None,None,None,None,None,South Reservation Handline 1 Maintenance,None,None,None,...,2024,2024,POLYGON,YES,06,California,CA,158144.83,39648525,"MULTIPOLYGON (((106414.462 -221037.937, 106416..."
35907,South Reservation Handline 2 Maintenance,None,None,None,None,None,South Reservation Handline 2 Maintenance,None,None,None,...,2024,2024,POLYGON,YES,06,California,CA,158144.83,39648525,"MULTIPOLYGON (((106670.586 -220987.059, 106682..."
35908,North Reservation Handline 1 Maintenance,None,None,None,None,None,North Reservation Handline 1 Maintenance,None,None,None,...,2024,2024,POLYGON,YES,06,California,CA,158144.83,39648525,"MULTIPOLYGON (((107209.35 -220737.845, 107213...."


In [22]:
#excel_out = pd.concat([enriched_points, enriched_lines, enriched_polygons])
#excel_out.drop(['geometry'], axis=1).to_csv('its_all.csv')

In [23]:
from datetime import datetime

In [24]:
def get_activity_report(enriched_points, enriched_lines, enriched_polygons):
    append_all = pd.concat([enriched_lines, enriched_points, enriched_polygons])
    append_all = append_all[(append_all['COUNTS_TO_MAS'] == 'YES')]
    
    append_all.geometry = gpd.points_from_xy(append_all['LONGITUDE'],append_all['LATITUDE'])
    
    
    append_all = append_all[["AGENCY",
        "ADMINISTERING_ORG",
        "PRIMARY_OWNERSHIP_GROUP",
        "COUNTY",
        "REGION",
        "ACTIVITY_DESCRIPTION",
        "ACTIVITY_CAT",
        "BROAD_VEGETATION_TYPE",
        "ACTIVITY_STATUS",
        "ACTIVITY_QUANTITY",
        "ACTIVITY_UOM",
        "ACTIVITY_END",
        "Year_txt",
        "geometry"
    ]]
    
    # check if geometry is_valid
    # personally this is redundent, unless lat, lon is not valid, but that would throw an error in previous part
    append_all = append_all[append_all.is_valid]
    
    
    def get_entity_type(agency):
        if agency in ['CALEPA', 'CALSTA', 'CNRA', 'PARKS', 'California State Parks']:
            return 'State'
        if agency in ['DOD', 'DOI', 'USDA', 'DOE', 'NPS']:
            return 'Federal'
        if agency in ['Industrial Timber', 'Timber Companies', 'TIMBER']:
            return 'Timber Companies'
        else:
            return None
        
        
    append_all['ENTITY_TYPE'] = append_all['AGENCY'].apply(get_entity_type)
    
    return append_all

In [25]:
activity_report_gdf = get_activity_report(enriched_points, enriched_lines, enriched_polygons)

In [26]:
save_gdf_to_gdb(activity_report_gdf, 
                report_path, 
                f'activity_report{datetime.today().strftime('%Y%m%d')}')

2025-07-18 12:40:51,797 INFO  [utils.save_gdf_to_gdb ]        Windows machine detected
2025-07-18 12:40:51,798 INFO  [utils.save_gdf_to_gdb ]        Check geodataframe geometry object and cast to Multi-x type if both exist
2025-07-18 12:40:51,831 INFO  [utils.save_gdf_to_gdb ]        Running GDAL OpenFileGDB to save to file


D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0\reports.gdb


2025-07-18 12:40:58,184 INFO  [pyogrio._io           ]  Created 498,157 records
2025-07-18 12:41:01,125 INFO  [utils.save_gdf_to_gdb ]        File saved to D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0\reports.gdb activity_report20250718


In [5]:
#activity_report_gdf = gpd.read_file(report_path, driver='OpenFileGDB', layer='activity_report20250714')

C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(


In [38]:
ifprs_path =  {'gdb_path': 'D:\\WORK\\wildfire\\Interagency-Tracking-System\\its\\ITSGDB_backup\\V2.0\\IFPRS_2023_2025.gdb',
  'layer_name': 'IFPRS_enriched_20250603'}

In [39]:
ifprs_gdf = gpd.read_file(ifprs_path['gdb_path'], driver='OpenFileGDB', layer=ifprs_path['layer_name'])

C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(


In [40]:
ifprs_gdf

,PROJECTID_USER,AGENCY,ORG_ADMIN_p,PROJECT_CONTACT,PROJECT_EMAIL,ADMINISTERING_ORG,PROJECT_NAME,PROJECT_STATUS,PROJECT_START,PROJECT_END,...,Source,Year,Year_txt,Act_Code,Crosswalk,Federal_FY,State_FY,TRMT_GEOM,COUNTS_TO_MAS,geometry
0,CADBR-HFR-FY24-MX-W,None,FWS,None,None,FWS,CADBR-HFR-FY24-MX-W,None,None,None,...,IFPRS,2023,2023,None,THIN_MECH,2024,2024,POLYGON,YES,"MULTIPOLYGON (((273163.046 -598315.334, 273154..."
1,CAR8R-BIL-FY23-MX-San Diego,None,FWS,None,None,FWS,CAR8R-BIL-FY23-MX-San Diego,None,None,None,...,IFPRS,2024,2024,None,MASTICATION,2024,2024,POLYGON,YES,"MULTIPOLYGON (((273495.062 -597705.931, 273500..."
2,NPS PWR CABR FY24 Defensible Space - BIL,None,NPS,None,None,NPS,NPS PWR CABR FY24 Defensible Space - BIL,None,None,None,...,IFPRS,2024,2024,None,BIOMASS_REMOVAL,2024,2025,POLYGON,YES,"MULTIPOLYGON (((259400.43 -589498.522, 259395...."
3,CATNR-HFR-FY24-MX-W,None,None,None,None,None,CATNR-HFR-FY24-MX-W,None,None,None,...,IFPRS,2024,2024,None,THIN_MECH,2024,2025,POLYGON,YES,"MULTIPOLYGON (((294989.891 -581637.748, 294979..."
4,CASOR-HFR-FY24-RX-N,None,None,None,None,None,CASOR-HFR-FY24-RX-N,None,None,None,...,IFPRS,2024,2024,None,BROADCAST_BURN,2024,2024,POLYGON,YES,"MULTIPOLYGON (((412018.092 -526655.842, 412041..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159,NPS PWR REDW FY24 Historical Structures-BIL,None,NPS,None,None,NPS,NPS PWR REDW FY24 Historical Structures-BIL,None,None,None,...,IFPRS,2024,2024,None,LOP_AND_SCAT,2024,2025,POLYGON,YES,"MULTIPOLYGON (((-329577.456 358902.421, -32958..."
160,NPS PWR REDW FY24 Hiouchi SFB - BIL,None,NPS,None,None,NPS,NPS PWR REDW FY24 Hiouchi SFB - BIL,None,None,None,...,IFPRS,2024,2024,None,MASTICATION,2024,2025,POLYGON,YES,"MULTIPOLYGON (((-339163.139 427016.036, -33916..."
161,CALKR-HFR-FY24-MX-W,None,FWS,None,None,FWS,CALKR-HFR-FY24-MX-W,None,None,None,...,IFPRS,2024,2024,None,LOP_AND_SCAT,2024,2024,POLYGON,YES,"MULTIPOLYGON (((-139124.742 440043.332, -13912..."
162,CALKR-HFR-FY24-CX-W,None,FWS,None,None,FWS,CALKR-HFR-FY24-CX-W,None,None,None,...,IFPRS,2024,2024,None,HERBICIDE_APP,2024,2025,POLYGON,YES,"MULTIPOLYGON (((-147598.282 443459.815, -14756..."


In [36]:
activity_report_gdf[activity_report_gdf.AGENCY.isna()]

,AGENCY,ADMINISTERING_ORG,PRIMARY_OWNERSHIP_GROUP,COUNTY,REGION,ACTIVITY_DESCRIPTION,ACTIVITY_CAT,BROAD_VEGETATION_TYPE,ACTIVITY_STATUS,ACTIVITY_QUANTITY,ACTIVITY_UOM,ACTIVITY_END,Year_txt,geometry,ENTITY_TYPE
35877,None,FWS,FEDERAL,SD,SOUTHERN_CA,THIN_MECH,MECH_HFR,GRASS_HERB,COMPLETE,38.0,AC,2023-11-02 00:00:00+00:00,2023,POINT (-117.095 32.592),None
35878,None,FWS,PRIVATE_NON-INDUSTRY,SD,SOUTHERN_CA,MASTICATION,MECH_HFR,GRASS_HERB,COMPLETE,15.0,AC,2024-01-19 00:00:00+00:00,2024,POINT (-117.092 32.592),None
35879,None,NPS,FEDERAL,SD,SOUTHERN_CA,BIOMASS_REMOVAL,MECH_HFR,SHRB_CHAP,COMPLETE,10.0,AC,2024-09-27 00:00:00+00:00,2024,POINT (-117.241 32.673),None
35880,None,None,PRIVATE_NON-INDUSTRY,SD,SOUTHERN_CA,THIN_MECH,MECH_HFR,SHRB_CHAP,COMPLETE,39.0,AC,2024-09-16 00:00:00+00:00,2024,POINT (-116.925 32.726),None
35881,None,None,LOCAL,IMP,SOUTHERN_CA,BROADCAST_BURN,PRESCRB_FIRE,AGRICULTURE,COMPLETE,280.0,AC,2024-06-12 00:00:00+00:00,2024,POINT (-115.689 33.091),None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36036,None,NPS,FEDERAL,HUM,NORTH_COAST,LOP_AND_SCAT,MECH_HFR,FOREST,COMPLETE,10.0,AC,2024-09-30 00:00:00+00:00,2024,POINT (-123.927 41.184),None
36037,None,NPS,PRIVATE_NON-INDUSTRY,DN,NORTH_COAST,MASTICATION,MECH_HFR,FOREST,COMPLETE,5.0,AC,2024-09-30 00:00:00+00:00,2024,POINT (-124.076 41.795),None
36038,None,FWS,FEDERAL,SIS,NORTH_COAST,LOP_AND_SCAT,MECH_HFR,AGRICULTURE,COMPLETE,172.0,AC,2024-06-26 00:00:00+00:00,2024,POINT (-121.729 41.957),None
36039,None,FWS,FEDERAL,SIS,NORTH_COAST,HERBICIDE_APP,MECH_HFR,WETLAND,COMPLETE,47.0,AC,2024-08-31 00:00:00+00:00,2024,POINT (-121.751 41.966),None


In [33]:
activity_report_gdf[activity_report_gdf.Year_txt == '2024'].groupby(['AGENCY', 'ADMINISTERING_ORG']).sum('ACTIVITY_QUANTITY')

ACTIVITY_QUANTITY
AGENCY ADMINISTERING_ORG                   
CALSTA CALTRANS               133459.499800
CNRA   CALFIRE                172012.204000
       CCC                      1783.460000
       CDFW                   103877.059607
       PARKS                   15087.253321
       SDRC                     3973.690000
       SMMC                      618.066657
       SNC                     12244.460000
       TAHOE                     304.500000
       WCB                      7671.460000
DOD    DOD                      2967.330000
DOE    DOE                      5482.900000
DOI    BIA                       260.000000
       BLM                     40467.823381
       FWS                       718.000000
TIMBER TIMBER                 187855.225804
USDA   USFS                   334071.200000

In [30]:
activity_report_gdf[(activity_report_gdf.ACTIVITY_DESCRIPTION == "AMW_AREA_RESTOR") | (activity_report_gdf.ACTIVITY_DESCRIPTION == "OAK_WDLND_MGMT")]

,AGENCY,ADMINISTERING_ORG,PRIMARY_OWNERSHIP_GROUP,COUNTY,REGION,ACTIVITY_DESCRIPTION,ACTIVITY_CAT,BROAD_VEGETATION_TYPE,ACTIVITY_STATUS,ACTIVITY_QUANTITY,ACTIVITY_UOM,ACTIVITY_END,Year_txt,geometry,ENTITY_TYPE
131462,CNRA,CDFW,STATE,MAD,SIERRA_NEVADA,OAK_WDLND_MGMT,MECH_HFR,FOREST,COMPLETE,3.680000,AC,2022-01-31 00:00:00+00:00,2022,POINT (56710.253 -88969.102),State
20390,TIMBER,TIMBER,PRIVATE_INDUSTRY,None,None,OAK_WDLND_MGMT,MECH_HFR,FOREST,COMPLETE,147.163751,AC,2021-12-31 00:00:00+00:00,2021,POINT (-284207.678 -48707.918),Timber Companies
20391,TIMBER,TIMBER,PRIVATE_INDUSTRY,None,None,OAK_WDLND_MGMT,MECH_HFR,FOREST,COMPLETE,280.000000,AC,2022-12-31 00:00:00+00:00,2022,POINT (-284207.678 -48707.918),Timber Companies
20392,TIMBER,TIMBER,PRIVATE_INDUSTRY,None,None,OAK_WDLND_MGMT,MECH_HFR,FOREST,COMPLETE,14.327502,AC,2023-12-31 00:00:00+00:00,2023,POINT (-284207.678 -48707.918),Timber Companies
20393,TIMBER,TIMBER,PRIVATE_INDUSTRY,None,None,OAK_WDLND_MGMT,MECH_HFR,FOREST,COMPLETE,0.000000,AC,2024-12-31 00:00:00+00:00,2024,POINT (-284207.678 -48707.918),Timber Companies
4574,CNRA,PARKS,STATE,HUM,NORTH_COAST,OAK_WDLND_MGMT,MECH_HFR,FOREST,COMPLETE,5.170592,AC,2023-08-24 00:00:00+00:00,2023,POINT (-124.03 40.335),State
4575,CNRA,PARKS,STATE,HUM,NORTH_COAST,OAK_WDLND_MGMT,MECH_HFR,FOREST,COMPLETE,0.241517,AC,2023-08-24 00:00:00+00:00,2023,POINT (-124.034 40.337),State
4577,CNRA,PARKS,STATE,HUM,NORTH_COAST,OAK_WDLND_MGMT,MECH_HFR,FOREST,COMPLETE,4.860597,AC,2023-08-25 00:00:00+00:00,2023,POINT (-124.039 40.34),State
4580,CNRA,PARKS,STATE,HUM,NORTH_COAST,OAK_WDLND_MGMT,MECH_HFR,FOREST,COMPLETE,1.676779,AC,2023-08-24 00:00:00+00:00,2023,POINT (-124.032 40.336),State
5427,CNRA,PARKS,STATE,HUM,NORTH_COAST,OAK_WDLND_MGMT,MECH_HFR,FOREST,COMPLETE,2.321638,AC,2023-08-24 00:00:00+00:00,2023,POINT (-124.033 40.337),State
